# Coding Setup

### Load Packages

In [92]:
# All packages (see requirements.txt)

## Data Collection and Processing
import pandas as pd
from collections import defaultdict

## System and Environment
import os
from dotenv import load_dotenv
import glob
import time

## Api calling
import requests
from opencage.geocoder import OpenCageGeocode

## Math operations
from math import radians, sin, cos, sqrt, atan2

### Set Working Environment

In [93]:
# ---insert your path here ---
os.chdir('/Users/janlinzner/Projects/thesis-spatial-seed-syndication') 

### Load Api-keys

Explanation about geocage and how to get an api-key

In [94]:
# load api keys from .env file or insert the key directly
load_dotenv()
api_key = os.getenv("geocage")

# Data Collection

### A: Companies

#### Companies: Load Data

Load all data files from the folders

In [95]:
root_directory = 'data/companies'

df_list = []

selected_columns = [
    "Organization Name", "Organization Name URL", "Description", "Industry Groups", "Headquarters Location",
    "Founded Date", "Number of Founders", "Last Equity Funding Type", "Exit Date" 
]

for subdir, _, _ in os.walk(root_directory):
    csv_files = glob.glob(os.path.join(subdir, '*.csv'))
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            df_list.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

final_df = pd.concat(df_list, ignore_index=True)

companies = final_df[selected_columns].copy()

Rename the variables

In [96]:
companies = companies.rename(columns={
    "Organization Name": "organization_name",
    "Organization Name URL": "organization_name_url",
    "Description": "organization_description",
    "Industry Groups": "industry_groups",
    "Headquarters Location": "hq_location",
    "Founded Date": "founded_date",
    "Number of Founders": "number_of_founders",
    "Last Equity Funding Type": "last_equity_funding_type",
    "Exit Date": "exit_date"
})

#### Companies: Clean Data

Convert data variables to date types

In [97]:
date_cols = ["founded_date"]
for col in date_cols:
    companies[col] = pd.to_datetime(companies[col], errors="coerce")

Get the founding year

In [98]:
companies["founded_year"] = companies["founded_date"].dt.year
companies.drop(columns=["founded_date"])

,organization_name,organization_name_url,organization_description,industry_groups,hq_location,number_of_founders,last_equity_funding_type,exit_date,founded_year
0,DePoly,https://www.crunchbase.com/organization/depoly,DePoly is a PET-to-raw-material recycling comp...,"Science and Engineering, Sustainability","Sion, Valais, Switzerland",3.0,Seed,NaN,2020
1,Hilo,https://www.crunchbase.com/organization/aktiia,Hilo is a healthcare company that provides blo...,"Artificial Intelligence (AI), Consumer Electro...","Neuchâtel, Neuchatel, Switzerland",3.0,Series B,NaN,2018
2,Sygnum,https://www.crunchbase.com/organization/sygnum,Sygnum is a digital asset banking group that e...,"Blockchain and Cryptocurrency, Financial Servi...","Zürich, Zurich, Switzerland",3.0,Series C,NaN,2017
3,HAYA Therapeutics,https://www.crunchbase.com/organization/haya-t...,HAYA Therapeutics is a biopharmaceutical compa...,"Biotechnology, Health Care, Science and Engine...","Lausanne, Vaud, Switzerland",2.0,Undisclosed,NaN,2017
4,Squirro,https://www.crunchbase.com/organization/squirro,Squirro is a software company that utilizes Au...,"Artificial Intelligence (AI), Data and Analyti...","Zürich, Zurich, Switzerland",4.0,Private Equity,NaN,2012
...,...,...,...,...,...,...,...,...,...
23093,OutdoorCompute,https://www.crunchbase.com/organization/outdoo...,OutdoorCompute's liquid immersion cooling tech...,NaN,"Benningbroek, Noord-Holland, The Netherlands",NaN,Seed,NaN,2023
23094,NEKOD,https://www.crunchbase.com/organization/nekod,NEKOD is a platform designed for business auto...,"Artificial Intelligence (AI), Data and Analyti...","Amsterdam, Noord-Holland, The Netherlands",NaN,Pre-Seed,NaN,2024
23095,Kvikk Insurance,https://www.crunchbase.com/organization/kvikk-...,Kvikk is a digital insurance platform.,Financial Services,"Bergen, Noord-Holland, The Netherlands",NaN,Pre-Seed,NaN,2021
23096,Dynamicpixels,https://www.crunchbase.com/organization/dynami...,"PaaS,Video games,Back-end","Gaming, Software","Delft, Zuid-Holland, The Netherlands",NaN,Pre-Seed,NaN,2023


Provide IDs

In [99]:
companies.reset_index(drop=True, inplace=True)
companies["company_id"] = companies.index + 1
companies["company_id"] = companies["company_id"].apply(lambda x: f"{x:06d}")
companies["company_id"] = companies["company_id"].astype("string")

cols = ["company_id"] + [col for col in companies.columns if col != "company_id"]
companies = companies[cols]

Number of founders feature with missing values = 0

In [100]:
companies["number_of_founders"] = companies["number_of_founders"].fillna(0).astype(int)

Create exit binary

In [101]:
companies.loc[:, "exit_binary"]  = companies["exit_date"].notna().astype(int)
companies["exit_binary"] = companies["exit_binary"].astype(bool)

Add geocoding and get longitude and latitude data

In [102]:
cache_file = "data/locations/location_cache.csv"

if os.path.exists(cache_file):
    cached_coords = pd.read_csv(cache_file)
else:
    cached_coords = pd.DataFrame(columns=["hq_location", "latitude", "longitude"])

unique_locations = companies["hq_location"].dropna().unique()
total = len(unique_locations)

cache_dict = dict(zip(cached_coords["hq_location"], zip(cached_coords["latitude"], cached_coords["longitude"])))

coords_list = []
new_cache_rows = []

for idx, location in enumerate(unique_locations, start=1):
    if location in cache_dict:
        lat, lng = cache_dict[location]
        row = {"hq_location": location, "latitude": lat, "longitude": lng}
        coords_list.append(row)
        print(f"[{idx}/{total}] (cached)  {location} → {lat}, {lng}")
    elif not location.strip():
        row = {"hq_location": location, "latitude": None, "longitude": None}
        coords_list.append(row)
        print(f"[{idx}/{total}] (blank)   {location!r} → None")
    else:
        params = {"key": api_key, "q": location, "limit": 1, "no_annotations": 1}
        try:
            resp = requests.get(base_url, params=params)
            data = resp.json()
            if data["status"]["code"] == 200 and data["results"]:
                lat = data["results"][0]["geometry"]["lat"]
                lng = data['results'][0]["geometry"]["lng"]
            else:
                lat = lng = None
        except Exception:
            lat = lng = None

        row = {"hq_location": location, "latitude": lat, "longitude": lng}
        coords_list.append(row)
        new_cache_rows.append(row)
        print(f"[{idx}/{total}] (fetched) {location} → {lat}, {lng}")
        time.sleep(0.5)

if new_cache_rows:
    new_cache_df = pd.DataFrame(new_cache_rows)
    cached_coords = pd.concat([cached_coords, new_cache_df], ignore_index=True)
    cached_coords = cached_coords.drop_duplicates(subset="hq_location", keep="first")
    cached_coords.to_csv(cache_file, index=False)

df_coords = pd.DataFrame(coords_list)

companies = companies.merge(df_coords, on="hq_location", how="left")

[1/2503] (cached)  Sion, Valais, Switzerland → 46.2311749, 7.3588795
[2/2503] (cached)  Neuchâtel, Neuchatel, Switzerland → 46.9895828, 6.9292641
[3/2503] (cached)  Zürich, Zurich, Switzerland → 47.3744489, 8.5410422
[4/2503] (cached)  Lausanne, Vaud, Switzerland → 46.5218269, 6.6327025
[5/2503] (cached)  Zug, Zug, Switzerland → 47.1679898, 8.5173652
[6/2503] (cached)  Sankt Gallen, Sankt Gallen, Switzerland → 47.425618, 9.3762397
[7/2503] (cached)  Kemptthal, Zurich, Switzerland → 47.4528517, 8.7058075
[8/2503] (cached)  Basel, Basel-Stadt, Switzerland → 47.5581077, 7.5878261
[9/2503] (cached)  Baar, Zug, Switzerland → 47.1951976, 8.5253985
[10/2503] (cached)  Bern, Bern, Switzerland → 46.9484742, 7.4521749
[11/2503] (cached)  Lugano, Ticino, Switzerland → 46.0050102, 8.9520281
[12/2503] (cached)  Zofingen, Aargau, Switzerland → 47.288491, 7.9458259
[13/2503] (cached)  Horgen, Zurich, Switzerland → 47.260692, 8.5976831
[14/2503] (cached)  Monthey, Valais, Switzerland → 46.252844, 6.94

Add the homecountry

In [103]:
companies["hq_country"] = companies["hq_location"].apply(lambda x: x.split(',')[-1].strip() if pd.notna(x) else None)

Add startup ecosystem binary

In [104]:
hotspot_data = [
    {"city": "London",       "lat": 51.5074, "lng": -0.1278},
    {"city": "Paris",        "lat": 48.8566, "lng": 2.3522},
    {"city": "Berlin",       "lat": 52.5200, "lng": 13.4050},
    {"city": "Stockholm",    "lat": 59.3293, "lng": 18.0686},
    {"city": "Munich",       "lat": 48.1351, "lng": 11.5820},
    {"city": "Helsinki",     "lat": 60.1695, "lng": 24.9354},
    {"city": "Madrid",       "lat": 40.4168, "lng": -3.7038},
    {"city": "Dublin",       "lat": 53.3498, "lng": -6.2603},
    {"city": "Tallinn",      "lat": 59.4370, "lng": 24.7536},
    {"city": "Copenhagen",   "lat": 55.6761, "lng": 12.5683},
    {"city": "Milan",        "lat": 45.4642, "lng": 9.1900},
    {"city": "Zurich",       "lat": 47.3769, "lng": 8.5417},
    {"city": "Oslo",         "lat": 59.9139, "lng": 10.7522},
    {"city": "Cambridge",    "lat": 52.2053, "lng": 0.1218},
    {"city": "Kyiv",         "lat": 50.4501, "lng": 30.5234},
    {"city": "Vienna",       "lat": 48.2082, "lng": 16.3738},
    {"city": "Brussels",     "lat": 50.8503, "lng": 4.3517},
    {"city": "Manchester",   "lat": 53.4808, "lng": -2.2426},
    {"city": "Lisbon",       "lat": 38.7169, "lng": -9.1399},
    {"city": "Prague",       "lat": 50.0755, "lng": 14.4378},
    {"city": "Warsaw",       "lat": 52.2297, "lng": 21.0122},
    {"city": "Hamburg",      "lat": 53.5511, "lng": 9.9937},
    {"city": "Oxford",       "lat": 51.7520, "lng": -1.2577},
    {"city": "Amsterdam",    "lat": 52.3676, "lng": 4.9041},
    {"city": "Barcelona",    "lat": 41.3851, "lng": 2.1734},
    {"city": "Lausanne",     "lat": 46.5197, "lng": 6.6323},
]

hotspots_df = pd.DataFrame(hotspot_data)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

def min_dist_to_hotspot(lat, lon):
    if pd.isna(lat) or pd.isna(lon):
        return float('inf')
    return hotspots_df.apply(
        lambda r: haversine(lat, lon, r["lat"], r["lng"]), axis=1
    ).min()

companies["latitude"]  = pd.to_numeric(companies["latitude"],  errors="coerce")
companies["longitude"] = pd.to_numeric(companies["longitude"], errors="coerce")

companies["distance_to_hub"] = companies.apply(
    lambda r: min_dist_to_hotspot(r["latitude"], r["longitude"]), axis=1
)
companies["hub_binary"] = (companies["distance_to_hub"] < 20).astype(bool)

Business orientation (B2B or B2C)

GICS Groups

In [105]:
companies["industry_groups"] = companies["industry_groups"].fillna('').str.replace(r'\s*,\s*', ',', regex=True)
dummies = companies["industry_groups"].str.get_dummies(sep=',').astype(bool)
companies = pd.concat([companies, dummies], axis=1)

In [106]:
to_move = [
    "Administrative Services", "Advertising", "Agriculture and Farming",
    "Apps", "Artificial Intelligence (AI)", "Biotechnology",
    "Blockchain and Cryptocurrency", "Clothing and Apparel",
    "Commerce and Shopping", "Community and Lifestyle",
    "Consumer Electronics", "Consumer Goods", "Content and Publishing",
    "Data and Analytics", "Design", "Education", "Energy", "Events",
    "Financial Services", "Food and Beverage", "Gaming",
    "Government and Military", "Hardware", "Health Care",
    "Information Technology", "Internet Services",
    "Lending and Investments", "Manufacturing", "Media and Entertainment",
    "Messaging and Telecommunications", "Mobile", "Music and Audio",
    "Natural Resources", "Navigation and Mapping", "Other", "Payments",
    "Platforms", "Privacy and Security", "Professional Services",
    "Real Estate", "Sales and Marketing", "Science and Engineering",
    "Social Impact", "Software", "Sports", "Sustainability",
    "Transportation", "Travel and Tourism", "Video"
]
cols_to_move = [c for c in to_move if c in companies.columns]

new_order = [c for c in companies.columns if c not in cols_to_move] + cols_to_move
companies = companies[new_order]

gics_map = {
    "Energy": "energy",
    "Blockchain and Cryptocurrency": "information_technology",
    "Natural Resources": "materials",
    "Administrative Services": "industrials",
    "Design": "industrials",
    "Government and Military": "industrials",
    "Manufacturing": "industrials",
    "Professional Services": "industrials",
    "Science and Engineering": "industrials",
    "Social Impact": "industrials",
    "Transportation": "industrials",
    "Clothing and Apparel": "consumer_discretionary",
    "Commerce and Shopping": "consumer_discretionary",
    "Community and Lifestyle": "consumer_discretionary",
    "Consumer Electronics": "consumer_discretionary",
    "Consumer Goods": "consumer_discretionary",
    "Education": "consumer_discretionary",
    "Events": "consumer_discretionary",
    "Sports": "consumer_discretionary",
    "Travel and Tourism": "consumer_discretionary",
    "Agriculture and Farming": "consumer_staples",
    "Food and Beverage": "consumer_staples",
    "Biotechnology": "health_care",
    "Health Care": "health_care",
    "Financial Services": "financials",
    "Lending and Investments": "financials",
    "Payments": "financials",
    "Apps": "information_technology",
    "Artificial Intelligence (AI)": "information_technology",
    "Blockchain and Cryptocurrency": "information_technology",
    "Data and Analytics": "information_technology",
    "Hardware": "information_technology",
    "Information Technology": "information_technology",
    "Internet Services": "information_technology",
    "Mobile": "information_technology",
    "Navigation and Mapping": "information_technology",
    "Platforms": "information_technology",
    "Privacy and Security": "information_technology",
    "Software": "information_technology",
    "Advertising": "communication_services",
    "Content and Publishing": "communication_services",
    "Gaming": "communication_services",
    "Media and Entertainment": "communication_services",
    "Messaging and Telecommunications": "communication_services",
    "Music and Audio": "communication_services",
    "Sales and Marketing": "communication_services",
    "Video": "communication_services",
    "Sustainability": "utilities",
    "Real Estate": "real_estate",
    "Other": "other",
}

head_to_orig = defaultdict(list)
for orig, head in gics_map.items():
    head_to_orig[head].append(orig)

for head, orig_cols in head_to_orig.items():
    companies[head] = companies[orig_cols].any(axis=1)

to_drop = [c for c in cols_to_move if gics_map[c] != c]
companies.drop(columns=to_drop, inplace=True)

gics_heads = list(head_to_orig.keys())
final_order = [c for c in companies.columns if c not in gics_heads] + gics_heads
companies = companies[final_order]

### B: Rounds

#### Rounds: Load Data

Load all rounds from the respective folders

In [108]:
root_directory = 'data/rounds'

selected_columns = [
    'Transaction Name URL', 'Organization Name', 'Organization Name URL', 'Funding Type', 'Money Raised (in USD)', 'Announced Date', 'Lead Investors', 'Investor Names'
]

df_list = []

for subdir, _, _ in os.walk(root_directory):
    csv_files = glob.glob(os.path.join(subdir, '*.csv'))
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            df_list.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

final_df = pd.concat(df_list, ignore_index=True)

rounds = final_df[selected_columns].copy()

#### Rounds: Clean Data

Drop duplicate rounds

In [109]:
rounds = rounds.drop_duplicates()

Rename the variables

In [110]:
rounds = rounds.rename(columns={
    "Transaction Name URL": "transaction_name_url",
    "Organization Name": "organization_name",
    "Organization Name URL": "organization_name_url",
    "Funding Type": "funding_type",
    "Money Raised (in USD)": "money_raised_usd",
    "Announced Date": "announced_date",
    "Lead Investors": "lead_investors",
    "Investor Names": "investor_names"
})

Filter out all other rounds than Pre-Seed or Seed

In [111]:
rounds = rounds[rounds['funding_type'].isin(['Pre-Seed', 'Seed'])]

Change announced date to date varialbe

In [112]:
rounds['announced_date'] = pd.to_datetime(rounds['announced_date'], errors='coerce')

Add IDs

In [113]:
rounds.reset_index(drop=True, inplace=True)
rounds['round_id'] = rounds.index + 1
rounds['round_id'] = rounds['round_id'].apply(lambda x: f"{x:06d}")
rounds['round_id'] = rounds['round_id'].astype('string')

cols = ['round_id'] + [c for c in rounds.columns if c != 'round_id']
rounds = rounds[cols]

### C: Investors

#### Investors: Load Data

Load all files from the respective folder

In [114]:
csv_dir = 'data/investor'
columns = [
    "Organization/Person Name", "Organization/Person Name URL", "Investor Type",
    "Location", "Description", "Founded Date", "CB Rank (Investor)"
]

files = glob.glob(os.path.join(csv_dir, '*.csv'))
dfs = [pd.read_csv(f) for f in files if os.path.getsize(f) > 0]

if dfs:
    investors = pd.concat(dfs, ignore_index=True)
    investors = investors[[col for col in columns if col in investors.columns]]
    investors.drop_duplicates(subset=["Organization/Person Name", "Organization/Person Name URL"], inplace=True)
else:
    investors = pd.DataFrame(columns=columns)

#### Investors: Clean Data

Rename the variables

In [115]:
investors = investors.rename(columns={
    "Organization/Person Name": "organization_person_name",
    "Organization/Person Name URL": "organization_person_name_url",
    "Investor Type": "investor_type",
    "Location": "hq_location",
    "Description": "description",
    "Founded Date": "founded_date",
    "CB Rank (Investor)": "cb_rank_investor"
})

Convert data features into the date type

In [116]:
date_cols = ["founded_date"]
for col in date_cols:
    investors[col] = pd.to_datetime(investors[col], errors='coerce')

Provide ID

In [117]:
investors.reset_index(drop=True, inplace=True)
investors['investor_id'] = investors.index + 1
investors['investor_id'] = investors['investor_id'].apply(lambda x: f"{x:06d}")
investors['investor_id'] = investors['investor_id'].astype('string')

cols = ['investor_id'] + [c for c in investors.columns if c != 'investor_id']
investors = investors[cols]

Add longitude and latitude pairs

In [118]:
cache_file = 'data/locations/location_cache.csv'

if os.path.exists(cache_file):
    cached_coords = pd.read_csv(cache_file)
else:
    cached_coords = pd.DataFrame(columns=['hq_location', 'latitude', 'longitude'])

unique_locations = investors['hq_location'].dropna().unique()
total = len(unique_locations)

cache_dict = dict(zip(cached_coords['hq_location'], zip(cached_coords['latitude'], cached_coords['longitude'])))

coords_list = []
new_cache_rows = []

for idx, location in enumerate(unique_locations, start=1):
    if location in cache_dict:
        lat, lng = cache_dict[location]
        row = {'hq_location': location, 'latitude': lat, 'longitude': lng}
        coords_list.append(row)
        print(f"[{idx}/{total}] (cached)  {location} → {lat}, {lng}")
    elif not location.strip():
        row = {'hq_location': location, 'latitude': None, 'longitude': None}
        coords_list.append(row)
        print(f"[{idx}/{total}] (blank)   {location!r} → None")
    else:
        params = {"key": api_key, "q": location, "limit": 1, "no_annotations": 1}
        try:
            resp = requests.get(base_url, params=params)
            data = resp.json()
            if data['status']['code'] == 200 and data['results']:
                lat = data['results'][0]['geometry']['lat']
                lng = data['results'][0]['geometry']['lng']
            else:
                lat = lng = None
        except Exception:
            lat = lng = None

        row = {'hq_location': location, 'latitude': lat, 'longitude': lng}
        coords_list.append(row)
        new_cache_rows.append(row)
        print(f"[{idx}/{total}] (fetched) {location} → {lat}, {lng}")
        time.sleep(0.5)

if new_cache_rows:
    new_cache_df = pd.DataFrame(new_cache_rows)
    cached_coords = pd.concat([cached_coords, new_cache_df], ignore_index=True)
    cached_coords = cached_coords.drop_duplicates(subset='hq_location', keep='first')
    cached_coords.to_csv(cache_file, index=False)

df_coords = pd.DataFrame(coords_list)

investors = investors.merge(df_coords, on='hq_location', how='left')

[1/2830] (cached)  Tokyo, Tokyo, Japan → 35.6812546, 139.766706
[2/2830] (cached)  London, England, United Kingdom → 51.4893335, -0.1440551
[3/2830] (cached)  Olten, Aargau, Switzerland → 47.3085666, 7.8932696
[4/2830] (cached)  Toronto, Ontario, Canada → 43.6534817, -79.3839347
[5/2830] (cached)  Muttenz, Basel-Landschaft, Switzerland → 47.525113, 7.6477401
[6/2830] (cached)  Mechelen-bovelingen, Limburg, Belgium → 50.7427937, 5.2629291
[7/2830] (cached)  Rome, Lazio, Italy → 41.8933203, 12.4829321
[8/2830] (cached)  Östermalm, Stockholms Lan, Sweden → 59.3382751, 18.0718928
[9/2830] (cached)  Mountain View, California, United States → 37.3893889, -122.0832101
[10/2830] (cached)  Charlotte, North Carolina, United States → 35.2272086, -80.8430827
[11/2830] (cached)  Bangalore, Karnataka, India → 12.9767936, 77.590082
[12/2830] (cached)  Vienna, Wien, Austria → 48.1857192, 16.4221587
[13/2830] (cached)  Hamburg, Hamburg, Germany → 53.550341, 10.000654
[14/2830] (cached)  Lille, Nord-Pas

Get homecountry

In [119]:
investors['hq_country'] = investors['hq_location'].apply(lambda x: x.split(',')[-1].strip() if pd.notna(x) else None)

Specific Investor Classification

Investor Type

In [120]:
def parse_types(x):
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return [t.strip().lower() for t in x]
    return [t.strip().lower() for t in str(x).split(",")]

keywords = [
    "accelerator",
    "incubator",
    "micro vc",
    "corporate venture capital",
    "angel group",
    "angel",
    "university program",
    "entrepreneurship program",
    "family investment office"
]

investors["type_list"] = investors["investor_type"].apply(parse_types)

for kw in keywords:
    col = kw.replace(" ", "_")
    investors[col] = investors["type_list"].apply(
        lambda types: any(kw in t for t in types)
    )

investors.drop(columns="type_list", inplace=True)

### Grants

#### Grants: Load Data

Load all grants from the respective folder

In [121]:
root_directory = 'data/grants'

df_list = []

selected_columns = [
    "Transaction Name", "Organization Name", "Organization Name URL", "Announced Date", "Money Raised (in USD)", "Investor Names"
]

for subdir, _, _ in os.walk(root_directory):
    csv_files = glob.glob(os.path.join(subdir, '*.csv'))
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            df_list.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

final_df = pd.concat(df_list, ignore_index=True)

grants = final_df[selected_columns].copy()

#### Grants: Data Cleaning

Renaming Variables

In [122]:
grants = grants.rename(columns={
    "Transaction Name": "transaction_name",
    "Organization Name": "organization_name",
    "Organization Name URL": "organization_name_url",
    "Announced Date": "announced_date",
    "Money Raised (in USD)": "money_raised_usd",
    "Investor Names": "investor_names"
})

Announced date into date type

In [123]:
cols = ["announced_date"]
for col in cols:
    grants[col] = pd.to_datetime(grants[col], errors='coerce')

# Data Engineering

Create a help table

In [124]:
seed_help = rounds[["round_id", "organization_name_url", "investor_names"]].copy()
rounds_investors = seed_help.dropna(subset=["investor_names"])

seed_help["investor_names"] = seed_help["investor_names"].str.split(",")
seed_help = seed_help.explode("investor_names")
seed_help["investor_names"] = seed_help["investor_names"].str.strip()

seed_help = seed_help[seed_help["investor_names"] != ""]

seed_help = seed_help.merge(companies[["organization_name_url", "company_id"]], on="organization_name_url", how="left")

seed_help.reset_index(drop=True, inplace=True)

seed_help = seed_help[seed_help["company_id"].notna()].reset_index(drop=True)

Only keep investors with an investor entry

In [125]:
seed_help = seed_help.merge(
    investors[["organization_person_name", "investor_id"]],
    left_on="investor_names",
    right_on="organization_person_name",
    how="inner"
)

seed_help = seed_help.drop(columns=["organization_person_name"])

Get the number of seed rounds of the company

In [126]:
seed_round_counts = rounds["organization_name_url"].value_counts()

companies["number_seed_rounds"] = companies["organization_name_url"].map(seed_round_counts).fillna(0).astype(int)

Get the number of investors

In [127]:
unique_investors_per_company = seed_help.groupby("company_id")["investor_id"].nunique()

companies["number_seed_investors"] = companies["company_id"].map(unique_investors_per_company).fillna(0).astype(int)

First and last date of seed round

In [128]:
first_round_date = rounds.groupby("organization_name_url")["announced_date"].min()
last_round_date = rounds.groupby("organization_name_url")["announced_date"].max()

companies["earliest_seed_round_date"] = companies["organization_name_url"].map(first_round_date)
companies["latest_seed_round_date"] = companies["organization_name_url"].map(last_round_date)

Year of first seed

In [129]:
companies["first_seed_round_year"] = companies["earliest_seed_round_date"].dt.year

Total seed funding in millions

In [130]:
company_seed_funding_m = (
    rounds.groupby("organization_name_url")["money_raised_usd"].sum() / 1_000_000
).reset_index()

company_seed_funding_m = company_seed_funding_m.rename(
    columns={"money_raised_usd": "total_seed_funding_m"}
)

companies = companies.merge(
    company_seed_funding_m, on="organization_name_url", how="left"
)

companies["total_seed_funding_m"] = companies["total_seed_funding_m"].fillna(0)

Low and high seed funding

In [131]:
companies["low_seed_funding"] = companies["total_seed_funding_m"] < 0.25 
companies["high_seed_funding"] = companies["total_seed_funding_m"] > 4.75

Follow on investor in seed binary

In [132]:
seed_help['follow_on_investor_in_seed_binary'] = (
    seed_help.groupby(['company_id', 'investor_id'])['round_id'].transform('count') > 1
)

follow_on_any = seed_help.groupby("company_id")["follow_on_investor_in_seed_binary"].any()
companies["follow_on_investor_in_seed_binary"] = companies["company_id"].map(follow_on_any).infer_objects(copy=False).fillna(False)


/var/folders/n8/ynbdx9r10lv6l8yh1m_hmbsh0000gn/T/ipykernel_47058/1171522059.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  companies["follow_on_investor_in_seed_binary"] = companies["company_id"].map(follow_on_any).infer_objects(copy=False).fillna(False)


Age

In [ ]:
companies["age_first_seed"] = companies["first_seed_round_year"] - companies["founded_year"]

One seed investor

In [135]:
companies["one_seed_investor_binary"] = (companies["number_seed_investors"] == 1)

Pre seed investor

In [136]:
pre_seed_companies = set(rounds.loc[rounds['funding_type'] == 'Pre-Seed', 'organization_name_url'])
companies['pre_seed_binary'] = companies['organization_name_url'].isin(pre_seed_companies)

Investor experience before the round

In [137]:
seed_help_dates = seed_help.merge(
    rounds[["round_id", "announced_date"]],
    on="round_id",
    how="left"
)

def count_prior_investments(row):
    investor_id = row["investor_id"]
    round_date = row["announced_date"]
    prior = seed_help_dates[
        (seed_help_dates["investor_id"] == investor_id) &
        (seed_help_dates["announced_date"] < round_date)
    ]
    return len(prior)

seed_help_dates["prior_seed_investments"] = seed_help_dates.apply(count_prior_investments, axis=1)

mean_prior_by_company = seed_help_dates.groupby("company_id")["prior_seed_investments"].mean()

companies["mean_prior_seed_investments_by_investors"] = companies["company_id"].map(mean_prior_by_company).fillna(0)

Large investor experience  
<sub>Large investor experience with `mean_prior_seed_investments_by_investors > 50` (approx. top 20 percentile)</sub>

In [138]:
companies["large_investor_experience"] = companies["mean_prior_seed_investments_by_investors"] > 50

Lead investor in seed

In [139]:
lead_investor_any = (
    rounds.groupby("organization_name_url")["lead_investors"]
    .apply(lambda x: x.notna() & (x.str.strip() != ""))
    .groupby("organization_name_url")
    .any()
)

companies["lead_investor_in_seed"] = companies["organization_name_url"].map(lead_investor_any).fillna(False)

/var/folders/n8/ynbdx9r10lv6l8yh1m_hmbsh0000gn/T/ipykernel_47058/3524646151.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  companies["lead_investor_in_seed"] = companies["organization_name_url"].map(lead_investor_any).fillna(False)


Investor with exits

In [140]:
company_exit_dates = companies.set_index("company_id")["exit_date"]

seed_help_dates = seed_help_dates.copy()
seed_help_dates["investor_exit_date"] = seed_help_dates["investor_id"].map(company_exit_dates)

def has_investor_with_prior_exit(company_id):
    rows = seed_help_dates[seed_help_dates["company_id"] == company_id]
    for _, row in rows.iterrows():
        exit_date = pd.to_datetime(row["investor_exit_date"], errors="coerce")
        if pd.notna(exit_date) and exit_date < row["announced_date"]:
            return True
    return False

companies["investor_with_exits"] = companies["company_id"].apply(has_investor_with_prior_exit)

All investors from the homecountry

In [141]:
def all_investors_from_homecountry(company_id):
    company_country = companies.loc[companies["company_id"] == company_id, "hq_country"].values[0]
    investor_countries = seed_help.loc[seed_help["company_id"] == company_id, "investor_id"].map(
        investors.set_index("investor_id")["hq_country"]
    )
    return investor_countries.notna().all() and (investor_countries == company_country).all()

companies["all_homecountry_investors_in_seed_binary"] = companies["company_id"].apply(all_investors_from_homecountry)

Received a grants before seed funding

In [142]:
earliest_seed_round_map = companies.set_index('company_id')['earliest_seed_round_date']

def has_grant_before_seed(row):
    company_id = companies.loc[companies['organization_name_url'] == row['organization_name_url'], 'company_id']
    if company_id.empty or pd.isna(row['announced_date']):
        return False
    earliest_date = earliest_seed_round_map.get(company_id.values[0], pd.NaT)
    return pd.notna(earliest_date) and row['announced_date'] < earliest_date

grants['grant_before_seed'] = grants.apply(has_grant_before_seed, axis=1)

grant_before_seed_map = grants[grants['grant_before_seed']].groupby('organization_name_url').size() > 0
companies['grant_before_seed'] = companies['organization_name_url'].map(grant_before_seed_map).fillna(False).astype(bool)

/var/folders/n8/ynbdx9r10lv6l8yh1m_hmbsh0000gn/T/ipykernel_47058/2446606564.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  companies['grant_before_seed'] = companies['organization_name_url'].map(grant_before_seed_map).fillna(False).astype(bool)


US investor

In [143]:
def has_us_investor(company_id):
    investor_ids = seed_help.loc[seed_help["company_id"] == company_id, "investor_id"]
    if investor_ids.empty:
        return False
    investor_countries = investors.set_index("investor_id").loc[investor_ids, "hq_country"]
    return (investor_countries == "United States").any()

companies["us_investor_binary"] = companies["company_id"].apply(has_us_investor)

Regional investor

In [144]:
def has_regional_seed_investor(company_id):
    company = companies.loc[companies["company_id"] == company_id]
    if company.empty or pd.isna(company.iloc[0]["latitude"]) or pd.isna(company.iloc[0]["longitude"]):
        return False
    lat1 = company.iloc[0]["latitude"]
    lon1 = company.iloc[0]["longitude"]
    investor_ids = seed_help.loc[seed_help["company_id"] == company_id, "investor_id"]
    if investor_ids.empty:
        return False
    investor_locs = investors.set_index("investor_id").loc[investor_ids, ["latitude", "longitude"]]
    investor_locs = investor_locs.dropna()
    if investor_locs.empty:
        return False
    for _, row in investor_locs.iterrows():
        lat2, lon2 = row["latitude"], row["longitude"]
        dist = haversine(lat1, lon1, lat2, lon2)
        if dist <= 100:
            return True
    return False

companies["regional_seed_investor_binary"] = companies["company_id"].apply(has_regional_seed_investor)

Same city region investor

In [145]:
def has_regional_seed_investor(company_id):
    company = companies.loc[companies["company_id"] == company_id]
    if company.empty or pd.isna(company.iloc[0]["latitude"]) or pd.isna(company.iloc[0]["longitude"]):
        return False
    lat1 = company.iloc[0]["latitude"]
    lon1 = company.iloc[0]["longitude"]
    investor_ids = seed_help.loc[seed_help["company_id"] == company_id, "investor_id"]
    if investor_ids.empty:
        return False
    investor_locs = investors.set_index("investor_id").loc[investor_ids, ["latitude", "longitude"]]
    investor_locs = investor_locs.dropna()
    if investor_locs.empty:
        return False
    for _, row in investor_locs.iterrows():
        lat2, lon2 = row["latitude"], row["longitude"]
        dist = haversine(lat1, lon1, lat2, lon2)
        if dist <= 30:
            return True
    return False

companies["city_seed_investor_binary"] = companies["company_id"].apply(has_regional_seed_investor)

Institutional regional investor

In [146]:
def is_institutional_investor(investor_id):
    url = investors.loc[investors["investor_id"] == investor_id, "organization_person_name_url"].values[0]
    return "organization" in url

def has_regional_institutional_investor(company_id):
    company = companies.loc[companies["company_id"] == company_id]
    if company.empty or pd.isna(company.iloc[0]["latitude"]) or pd.isna(company.iloc[0]["longitude"]):
        return False
    lat1 = company.iloc[0]["latitude"]
    lon1 = company.iloc[0]["longitude"]
    investor_ids = seed_help.loc[seed_help["company_id"] == company_id, "investor_id"]
    if investor_ids.empty:
        return False
    investor_locs = investors.set_index("investor_id").loc[investor_ids, ["latitude", "longitude", "organization_person_name_url"]]
    investor_locs = investor_locs.dropna()
    if investor_locs.empty:
        return False
    for _, row in investor_locs.iterrows():
        if "organization" in row["organization_person_name_url"]:
            lat2, lon2 = row["latitude"], row["longitude"]
            dist = haversine(lat1, lon1, lat2, lon2)
            if dist <= 100:
                return True
    return False

companies["regional_seed_investor_institutional_binary"] = companies["company_id"].apply(has_regional_institutional_investor)

Success definition

In [147]:
success_types = ['Series A', 'Series B', 'Venture - Series Unknown', 'Private Equity',
 'Series C', 'Corporate Round', 'Series D', 'Series E', 'Series F', 'Series G', 'Series I']


companies["success"] = (
    companies["last_equity_funding_type"].isin(success_types) | companies["exit_binary"]
)

In [159]:
success_types = ['Series A', 'Series B', 'Venture - Series Unknown', 'Private Equity',
 'Series C', 'Corporate Round', 'Series D', 'Series E', 'Series F', 'Series G', 'Series I']


companies["post_seed_success"] = (
    companies["last_equity_funding_type"].isin(success_types)
)

Investor types

In [155]:
for kw in keywords:
    col = kw.replace(" ", "_")
    investor_ids_with_type = set(investors.loc[investors[col], "investor_id"])
    companies[col + "_seed_investor_binary"] = companies["company_id"].isin(
        seed_help.loc[seed_help["investor_id"].isin(investor_ids_with_type), "company_id"]
    )

# Scope 

Remove all companies with 0 investment rounds

In [148]:
companies = companies[companies["number_seed_rounds"] > 0].reset_index(drop=True)

Remove blockchain companies

In [149]:
companies = companies[~companies["industry_groups"].str.split(",").apply(lambda x: "Blockchain and Cryptocurrency" in x)].reset_index(drop=True)

Only companies founded between 2007 and 2020

In [150]:
companies = companies[companies["founded_year"] < 2021].reset_index(drop=True)

At least one investor with coordinates

In [151]:
valid_investors = seed_help.merge(
    investors[['investor_id', 'latitude', 'longitude']],
    on='investor_id',
    how='left'
)

valid_investors = valid_investors.dropna(subset=['latitude', 'longitude'])

companies_with_valid_investors = valid_investors['company_id'].unique()

companies = companies[companies['company_id'].isin(companies_with_valid_investors)].reset_index(drop=True)

At least one funding round

In [152]:
companies = companies[companies["number_seed_rounds"] > 0].reset_index(drop=True)

First seed year between 2007 and 2020

In [ ]:
companies = companies.copy()

companies = df[
    companies["first_seed_round_year"].between(2007, 2020, inclusive="both")
].reset_index(drop=True)

# Output

In [161]:
output_path = 'data/regression_sets/companies.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True) 
companies.to_csv(output_path, index=False)